<a href="https://colab.research.google.com/github/tt35/NLP_highlighting/blob/main/2021_word_difficulty_%26_color_coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Highlighting words based on the difficulty of the words in the context.

In [ ]:
!pip install transformers 

     |████████████████████████████████| 2.6 MB 21.8 MB/s 
     |████████████████████████████████| 636 kB 39.3 MB/s 
     |████████████████████████████████| 895 kB 41.3 MB/s 
     |████████████████████████████████| 3.3 MB 32.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from torch import tensor
from IPython.display import HTML

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("distilgpt2", add_prefix_space=True) # smaller version of GPT-2
# Alternative to add_prefix_space is to use `is_split_into_words=True`
# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained("distilgpt2", pad_token_id=tokenizer.eos_token_id)

In [ ]:
def pred_words(phrase):
  i = 0
  j = 0
  difficulty_list = []
  htmlstring = phrase.split()[0]

  phrase_token_ids = tokenizer.encode(phrase)
  output = model.forward(tensor([phrase_token_ids]))
  logit = output.logits[0]

  # Generate tokesn for each word except for the first word in the phrase. Compare the actual word with the predicted words. 
  # If the actual word is one of the predicted words, append "True" to the list. If not, append "False" to the list.
  while i < len(phrase.split()) - 1:
    logits_for_next_token = logit[i]
    # top 50,000 words
    likely_token_indicies = logits_for_next_token.topk(50000).indices
    predicted_words = tokenizer.decode(likely_token_indicies)
    # top 20 words
    top_predicted_words = predicted_words[:10]
    # actual word
    actual_word = phrase.split()[i + 1]

    # classify words based on difficulty
    if actual_word not in predicted_words:
      difficulty_list.append("hard")
    elif actual_word in top_predicted_words:
      difficulty_list.append("easy")
    else:
      difficulty_list.append("neutral")
    i += 1
  
  # Look at the boolean list, and highlight the words based on difficulty
  while j < len(phrase.split()) - 1:
    if difficulty_list[j] == "hard":
      htmlstring += " " + "<span style=\"color: red;\">" + phrase.split()[j + 1] + "</span>"
    if difficulty_list[j] == "easy":
      htmlstring += " " + "<span style=\"color: blue;\">" + phrase.split()[j + 1] + "</span>"
    if difficulty_list[j] == "neutral":
      htmlstring += " " + "<span>" + phrase.split()[j + 1] + "</span>"
    j += 1

  return htmlstring

In [ ]:
raw_html = pred_words("To give you an idea of effective communication between businesses and customers, here are eight sample text messages to customers that will ensure you deliver value to each recipient.")

In [ ]:
HTML(raw_html)

# 7/23/2021 Fixed model



In [ ]:
# Generate tokesn for each word except for the first word in the phrase. Compare the actual word with the predicted words. 
# If the actual word is one of the predicted words, append "True" to the list. If not, append "False" to the list.
def get_word_difficulty(indecies, logit, ids_to_tokens):
  difficulty_list = []
  for i in indecies:
      logits_for_next_token = logit[i]
      # neutral-frequency
      likely_token_indicies = logits_for_next_token.topk(2000).indices
      predicted_words = tokenizer.decode(likely_token_indicies)
      # high-frequency
      top_predicted_words = predicted_words[:300]
    
      # classify words based on difficulty
      if (ids_to_tokens[i]) not in predicted_words:
        difficulty_list.append("hard")
      elif (ids_to_tokens[i]) in top_predicted_words:
        difficulty_list.append("easy")
      else:
        difficulty_list.append("neutral")
      
      word_and_difficulty = zip(ids_to_tokens, difficulty_list)
      word_and_difficulty = list(word_and_difficulty)
      
  return word_and_difficulty  

In [ ]:
# Look at the boolean list, and highlight the words based on difficulty
def get_htmlstring(word_and_difficulty):
  j = 0
  htmlstring = ""
  for j in word_and_difficulty:
    if j[1] == "hard":
      htmlstring += " " + "<span style=\"color: red;\">" + j[0] + "</span>"
    if j[1] == "easy":
      htmlstring += " " + "<span style=\"color: blue;\">" + j[0] + "</span>"
    if j[1] == "neutral":
      htmlstring += " " + "<span>" + j[0] + "</span>"

  return htmlstring

In [ ]:
# Highlight words based on difficulty.
def pred_words_2(phrase):
  phrase_token_ids = tokenizer.encode(phrase)
  output = model.forward(tensor([phrase_token_ids]))
  logit = output.logits[0]

  # make a list of indices
  indecies_list = list(enumerate(phrase_token_ids))
  indecies = [i[0] for i in indecies_list]

  # convert ids to tokens
  ids_to_tokens = [token.replace('Ġ', ' ') for token in tokenizer.convert_ids_to_tokens(phrase_token_ids)]

  # call functions
  word_and_difficulty =  get_word_difficulty(indecies, logit, ids_to_tokens)
  htmlstring = get_htmlstring(word_and_difficulty)

  return htmlstring

In [ ]:
raw_html_2 = pred_words_2("""A small helicopter took off from the surface of Mars on April 19. It hovered about 3 meters (10 feet) above the Martian surface for approximately 30 seconds before touching back down. This was the first time humans have ever flown an aircraft on another planet. The helicopter, named Ingenuity, arrived on Mars in February 2021. It traveled to the red planet in the belly of NASA's Perseverance rover, which is a moving robot that will collect data, or information, about Mars.""")

In [ ]:
HTML(raw_html_2)

# Restricted Model

Model that sees only two previous tokens.

In [ ]:
# For the last word in the sets of two or three words, add the difficulty of the last word.
def get_restricted_htmlstring(word_and_difficulty, phrase):
  htmlstring = ""
  for index, item in enumerate(word_and_difficulty):
    if index == len(list(enumerate(word_and_difficulty))) - 1:
      if item[1] == "hard":
        htmlstring += " " + "<span style=\"color: red;\">" + item[0] + "</span>"
      if item[1] == "easy":
        htmlstring += " " + "<span style=\"color: blue;\">" + item[0] + "</span>"
      if item[1] == "neutral":
        htmlstring += " " + "<span>" + item[0] + "</span>"

  return htmlstring

In [ ]:
# Highlight words based on difficulty.
def pred_words_3(phrase):
  phrase_token_ids = tokenizer.encode(phrase)
  output = model.forward(tensor([phrase_token_ids]))
  logit = output.logits[0]

  # make a list of indices
  indecies_list = list(enumerate(phrase_token_ids))
  indecies = [i[0] for i in indecies_list]

  # convert ids to tokens
  ids_to_tokens = [token.replace('Ġ', '') for token in tokenizer.convert_ids_to_tokens(phrase_token_ids)]
  
  # call functions
  word_and_difficulty =  get_word_difficulty(indecies, logit, ids_to_tokens)
  htmlstring = get_restricted_htmlstring(word_and_difficulty, phrase)

  return htmlstring

In [ ]:
import re

# slice the input by space and punctuations
phrase = """A small helicopter took off from the surface of Mars on April 19.."""

phrase_token_ids = tokenizer.encode(phrase)
ids_to_tokens = [token.replace('Ġ', ' ' ) for token in tokenizer.convert_ids_to_tokens(phrase_token_ids)]

# string including the first word of the input
restricted_htmlstring = ids_to_tokens[0] + " "

for i in range(1, len(ids_to_tokens)):
  if i  < 2:
   sliced_texts = ''.join(ids_to_tokens[i-1:i+1])
  else:
   sliced_texts = ''.join(ids_to_tokens[i-2:i+1])
  i += 1
  html_for_last_word = pred_words_3(sliced_texts)
  restricted_htmlstring += html_for_last_word

In [ ]:
# result of the restricted model that can only see the previous two tokens
HTML(restricted_htmlstring)

In [ ]:
# result of the unrestricted model
HTML(raw_html_2)

# Model Comparison (Works in Progress)

In [ ]:
# http://stephantul.github.io/python/pytorch/2020/09/18/fast_topk/

def get_top_k(x, indices):
   vals = x[range(len(x)), indices]
   return (x > vals[:, None]).long().sum(1)

In [ ]:
torch.manual_seed(44)
probas = torch.rand(32, 50000)
probas /= probas.sum(1)[:, None]
targets = torch.randint(0, 50000, (32,))